<a href="https://colab.research.google.com/github/lenahdtr/fallstudie_lkws/blob/main/Stufe3l2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie- LKW Zuteilung - Stufe 3.2


In [1]:
! pip install -q pyscipopt

In [2]:
import pandas as pd
from pyscipopt import Model, quicksum

# Daten laden

*Daten aus den Tabellen, die hard codiert werden:*

In [3]:
routes_data = [
    {"route_id": "t-4", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.28125, "endtime": 0.71875},
    {"route_id": "t-5", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.2708333333333333, "endtime": 0.7083333333333334},
    {"route_id": "t-6", "distance_total": 250.0, "distance_toll": 150.0, "starttime": 0.25, "endtime": 0.6875},

    {"route_id": "s-1", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "s-2", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "s-3", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.375, "endtime": 0.6666666666666666},
    {"route_id": "s-4", "distance_total": 120.0, "distance_toll": 32.0, "starttime": 0.2708333333333333, "endtime": 0.6875},

    {"route_id": "w1", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w2", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3333333333333333, "endtime": 0.75},
    {"route_id": "w3", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.28125, "endtime": 0.6979166666666666},
    {"route_id": "w4", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.25, "endtime": 0.6666666666666666},
    {"route_id": "w5", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.2916666666666667, "endtime": 0.7083333333333334},
    {"route_id": "w6", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.22916666666666666, "endtime": 0.6458333333333334},
    {"route_id": "w7", "distance_total": 100.0, "distance_toll": 32.0, "starttime": 0.3020833333333333, "endtime": 0.71875},

    {"route_id": "r1", "distance_total": 285.0, "distance_toll": 259.0, "starttime": 0.75, "endtime": 0.9375},
    {"route_id": "r2", "distance_total": 250.0, "distance_toll": 220.0, "starttime": 0.6875, "endtime": 0.90625},
    {"route_id": "r3", "distance_total": 235.0, "distance_toll": 219.0, "starttime": 0.7395833333333334, "endtime": 0.8958333333333334},

    {"route_id": "h3", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.78125, "endtime": 0.9479166666666666},
    {"route_id": "h4", "distance_total": 180.0, "distance_toll": 160.0, "starttime": 0.7708333333333334, "endtime": 0.9375},

    {"route_id": "k1", "distance_total": 275.0, "distance_toll": 235.0, "starttime": 0.6875, "endtime": 0.9375},
]
routes_df = pd.DataFrame(routes_data)

# Initialisierung des Modells

In [4]:
model = Model("Stufe3_Diesel-Elektro_Optimierung")

# 1. Indexmengen

Tour

In [5]:
ROUTEN = routes_df.index.tolist()   # Menge aller Touren

In [6]:
FUHRPARK = range(1, len(ROUTEN)+1)  # Alle geleasten LKWs im Fuhrpark

In [7]:
FTypen = ["Diesel", "E400", "E600"] # Verfügbare LKW-Typen

In [8]:
ZEIT = range(0, 96)  # Zeitschritte in 15-Minuten-Intervallen

In [9]:
STypen = ["Alpitronic-50", "Alpitronic-200", "Alpitronic-400"]

In [10]:
LADEPUNKT = [1, 2]

In [11]:
SÄULENBESTAND = [1, 2, 3]

# 2. Parameter

LKW

In [12]:
# Investitonskosten von LKW-Typ t
C_CAPEX_LKW = {"Diesel": 24000.0,
           "E400": 50000.0,
           "E600": 60000.0
}

# Operative Kosten von LKW-Typ t
C_OPEX_LKW = {"Diesel": 6000.0,
          "E400": 5000.0,
          "E600": 6000.0
}

# Engerieverbrauch der LKW-Typ t
VERBRAUCH = {"Diesel": 26.0/100,
             "E400":  20/100, #105.0/100,
             "E600": 20/100 #110.0/100
}

# Batteriekapazität
BATTERIE = {"Diesel": 0.0,
            "E400": 414.0,
            "E600": 621.0
            }

# Steuerkosten von Fahrzeugtyp t
C_STEUER = {"Diesel": 556.0,
            "E400": 0.0,
            "E600": 0.0
}

# Erlös THG von Fahrzeugtyp t
THG = {"Diesel": 0.0,
       "E400": 1000.0,
       "E600": 1000.0
}

# Power Limit, max. Ladeleistung von LKW-Typ t
POWER_LIMIT_LKW = {"Diesel": 0.0,
          "E400": 400.0,
          "E600": 400.0
}

Preise

In [13]:
# Kraftstoffpreis
P_DIESEL = 1.60

# Stromkosten pro kwH
P_STROM = 0.25

Tour

In [14]:
#Gesamte Fahrstrecke der Tour r
DISTANZ_GESAMT = {r: routes_df.loc[r, 'distance_total'] for r in ROUTEN}  #Wir schauen in jeder Spalte Gesamtdistanz in der Tabelle von den Routen und nehmen das für jede ROute raus

#Mautpflichtige STrecke der Tour r
DISTANZ_MAUT = {r: routes_df.loc[r, 'distance_toll'] for r in ROUTEN}  #siehe oben

# Mautgebühr
C_MAUT = 0.34

Zeit

In [15]:
#Startzeitpunkt der Tour r
START = {r: routes_df.loc[r, 'starttime'] * 96 for r in ROUTEN}

#Endzeitpunkt der Tour r
END = {r: routes_df.loc[r, 'endtime'] * 96 for r in ROUTEN}

#Einsatztage pro Jahr
N_TAGE = 260

#Hilfsvariable ->läuft Route r zu welchen zeitpunkt z
from collections import defaultdict
ROUTE_AKTIV = defaultdict(int)
for r in ROUTEN:
    for z in ZEIT:
        ROUTE_AKTIV[r, z] = 1 if START[r] <= z < END[r] else 0    #Ende kommt raus, weil sonst haben wir anstatt 27 28 Slots (nahc ENde kommt ja kein Slot mehr) + verbruacht zu viel Energie

ROUTE_DAUER = {
    r: sum(ROUTE_AKTIV[r,z] for z in ZEIT)
    for r in ROUTEN
}


Ladeinfrastrukut

In [16]:
# Jährliche Kosten Ladeinfrastruktur (Capex + Opex)
C_CAPEX_S = {
    "Alpitronic-50": 3000.0,
    "Alpitronic-200": 10000.0,
    "Alpitronic-400": 16000.0
}

C_OPEX_S = {
    "Alpitronic-50": 1000.0,
    "Alpitronic-200": 1500.0,
    "Alpitronic-400": 2000.0
}

# Power Limit, max. Ladeleistung vom Säulentyp S
POWER_LIMIT_S = {"Alpitronic-50": 50.0,
        "Alpitronic-200": 200.0,
        "Alpitronic-400": 400.0
}

C_GEBÜHR = 1000


#Anzahl Ladepunkte pro Säule
#LADEPUNKTE = {"Alpitronic-50": 2,
#      "Alpitronic-200": 2,
#      "Alpitronic-400": 2
#}

# 3. Entscheidungsvariablen

LKW

In [17]:
# a[lkw, tour]: 1, wenn LKW f die Tour r fährt
lkw_fährt_r = {}
for f in FUHRPARK:
    for r in ROUTEN:
        lkw_fährt_r[f, r] = model.addVar(vtype="B", name=f"LKW_fährt_Route{f}_{r}")

In [18]:
# y[lkw, typ]: 1, wenn LKW f den Typ t hat
lkw_hat_typ = {}
for f in FUHRPARK:
    for t in FTypen:
        lkw_hat_typ[f, t] = model.addVar(vtype="B", name=f"LKW_hat_Typ_{f}_{t}")

In [19]:
# Bedeutung: LKW f vom Typ t fährt Route r (z[f,r,t] ersetzt das Produkt (a * y))
lkw_fährt_Route_mit_typ = {}
for f in FUHRPARK:
   for r in ROUTEN:
       for t in FTypen:
           lkw_fährt_Route_mit_typ[f, r, t] = model.addVar(vtype="B", name=f"LKW_fährt_Route_mit_Type{f}_{r}_{t}")


In [20]:
# LKW f lädt  zum Zeitpunkt z an Säule s an Ladepunkt l
lkw_lädt = {}
for f in FUHRPARK:
    for z in ZEIT:
        for b in SÄULENBESTAND:
            for l in LADEPUNKT:
              lkw_lädt[f, z, b, l] = model.addVar(vtype="B", name=f"LKW_lädt_{f}_{z}_{b}_{l}")

In [21]:
# Batteriestand LKW f zum Zeitpunkt z
soc = {}
for f in FUHRPARK:
    for z in ZEIT:
        soc[f, z] = model.addVar(vtype="C", lb=0, name=f"soc_{f}_{z}")

In [22]:
# x[typ]: Anzahl der LKWs f pro Typ t
anzahl_lkw = {}
for t in FTypen:
    anzahl_lkw[t] = model.addVar(vtype="I", name=f"anzahl_LKW_{t}")

Ladeinfrastruktur

In [23]:
# n[s]: Anzahl der installierten Ladesäulen vom Typ s
anzahl_Säule = {}
for s in STypen:
    anzahl_Säule[s] = model.addVar(vtype="I", lb=0, name=f"anzahl_Säule_{s}")

In [24]:
# Säule hat Typ
# y[lkw, typ]: 1, wenn LKW f den Typ t hat
säule_hat_typ = {}
for b in SÄULENBESTAND:
    for s in STypen:
        säule_hat_typ[b, s] = model.addVar(vtype="B", name=f"Säule_hat_Typ_{b}_{s}")

In [25]:
# Welche Leistung nimmt welcher Ladepunkt l von der Ladesäule b zum Zeitpunkt z
# maximale Netzleistung von dem Ladepunkt
leistung_ladepunkt_von_säule ={}
for z in ZEIT:
  for b in SÄULENBESTAND:
      for l in LADEPUNKT:
        leistung_ladepunkt_von_säule[z, b, l] = model.addVar(vtype="I", name=f"Leistung_von_Ladepunkt_von_Säule_{z}_{b}_{l}")

In [26]:
# Welche Leistung nimmt welcher Ladepunkt l von der Ladesäule b zum Zeitpunkt z
# wie viel Leistung ziehe ich aktuell aus dem Stromnetz
leistung_lkw_von_ladepunkt ={}
for f in FUHRPARK:
  for z in ZEIT:
    for b in SÄULENBESTAND:
      for l in LADEPUNKT:
        leistung_lkw_von_ladepunkt[f, z, b, l] = model.addVar(vtype="I", name=f"Leistung_LKW_von_Ladepunkt_{f}_{z}_{b}_{l}")

# 4. Zielfunktion

In [27]:
# TEIL A: FIXKOSTEN (Was uns die LKWs pro Jahr kosten, egal wie viel sie fahren)
# Wir summieren für jeden Typ (Diesel, E400, E600):
# Grundgebühr Strom + (Anzahl LKWs) * (Leasing + Wartung + Steuer - THG-Bonus)
fix_kosten_lkw = quicksum(lkw_hat_typ[f, t] * (C_CAPEX_LKW[t] + C_OPEX_LKW[t] + C_STEUER[t] - THG[t]) for f in FUHRPARK for t in FTypen)
fix_kosten_lkw = fix_kosten_lkw + C_GEBÜHR

# TEIL B: VARIABLE KOSTEN (Sprit/Strom & Maut insgesamt für alle Tage)
variable_kosten_total = quicksum(
    # Diesel Kosten
    lkw_fährt_Route_mit_typ[f, r, "Diesel"] * (DISTANZ_GESAMT[r] * VERBRAUCH["Diesel"] * P_DIESEL + DISTANZ_MAUT[r] * C_MAUT) +

    # E400 Kosten
    lkw_fährt_Route_mit_typ[f, r, "E400"] * DISTANZ_GESAMT[r] * VERBRAUCH["E400"] * P_STROM +

    # E600 Kosten
    lkw_fährt_Route_mit_typ[f, r, "E600"] * DISTANZ_GESAMT[r] * VERBRAUCH["E600"] * P_STROM
    for f in FUHRPARK for r in ROUTEN
)

# Kosten Ladeninfrastruktur
kosten_ladeinfrastruktur = quicksum( anzahl_Säule[s] * (C_CAPEX_S[s] + C_OPEX_S[s]) for s in STypen )

# GESAMTZIEL: Fixkosten + Variable Kosten (summiert über alle Tage) + Kosten Ladeinfrastruktur
model.setObjective(
    fix_kosten_lkw + variable_kosten_total + kosten_ladeinfrastruktur,
    "minimize")

# 5. Nebenbedingungen

LKW

Flotten-*Logik*

In [28]:
# NB 1: LINEARE NEBENBEDINGUNGEN (damit gilt a[f,r] = 1  ⇔  Σt z[f,r,t] ≥ 1)
for f in FUHRPARK:
     for r in ROUTEN:
         for t in FTypen:
                # wenn lkw_fährt_Route_mit_typ =1, dann muss lkw_fährt_r 1        (wenn ===, dann gilt das auch gleichzeitig mit allen Typen)
            model.addCons(
                lkw_fährt_Route_mit_typ[f, r, t] <= lkw_fährt_r[f, r]
                )

                # wenn lkw_fährt_Route_mit_typ =1, dann muss lkw_hat_typ 1
            model.addCons(
                lkw_fährt_Route_mit_typ[f, r, t] <= lkw_hat_typ[f, t],
                          name=f"Link_zfdrt_y_{f}_{r}_{t}")

# Ensure a[f,r] is 1 if route r is driven by truck f
for f in FUHRPARK:
    for r in ROUTEN:
        model.addCons(
            lkw_fährt_r[f, r] <= quicksum(lkw_fährt_Route_mit_typ[f, r, t] for t in FTypen),
            name=f"Activate_a_{f}_{r}")


In [29]:
# NB 2: Tourenabdeckung -> Jede Tour muss genau einmal gefahren werden
for r in ROUTEN:
    model.addCons(
        quicksum(lkw_fährt_r[f, r] for f in FUHRPARK) == 1
        )

In [30]:
# NB 3: Typ-Eindeutigkeit -> Ein LKW f muss genau einem Typ entsprechen
for f in FUHRPARK:
    model.addCons(
        quicksum(lkw_hat_typ[f, t] for t in FTypen) <= 1          #wenn wir das nicht haben, kauft das Programm "Geister-LKWs" weil die nichts kosten weil wir wollen kosten minimieren (Unsere Kosten an den Typ gekoppelt, wenn LKW keinen Typ hat, hat er auch keine Kosten)
        )

In [31]:
# NB 4: Kauf-Kopplung (Fahren nur bei Kauf) -> LKW (ein bisschen redundant, weil jeder LKW einem Typ zugewiesen wird, aber drinne lassen, damit Schlupflöcher weg)
#for f in FUHRPARK:
 #       model.addCons(
  #          quicksum(lkw_fährt_r[f, r] for r in ROUTEN) <= len(ROUTEN) * quicksum(lkw_hat_typ[f, t] for t in FTypen)        # Wenn rechts d.h. y=0 ist dann hat LKW nicht Typ t, aber dann muss links a (=LkW fährt TOUR) auch 0 sein also LKW fährt nicht weil <=
   #         )

In [32]:
# NB 5: Zeit-Check (Keine Tour-Überlappung pro LKW)
for f in FUHRPARK: #für jeden LKW im FUhrpark
    for r in ROUTEN:  #(+nächste Zeile:) schaue mir jedes Paar von den Routen an also r=Route1 und k = Route2
        for k in ROUTEN:
          # Dass jedes Paar nur einmal geprüft wird
          if r < k and START[k] < END[r] and START[r] < END[k]:       #Route k startet bevor Route r endet UND Route r startet bevor Route k endet
                    model.addCons(
                        lkw_fährt_r[f, r] + lkw_fährt_r[f, k] <= 1
                        )

In [33]:
# NB 6: Flottenberechnung -> Anzahl LKWs pro Typ ist genauso groß wie die SUmme aus allen Fahrzeugen die einen Typ haben
for t in FTypen:
    model.addCons(
        anzahl_lkw[t] == quicksum(lkw_hat_typ[f, t] for f in FUHRPARK)
        )


Ladeinfrastruktur

*allgemein*




In [34]:
#!!

# NB 15: Jede Säule hat höchstens einen Säulentypen
for b in SÄULENBESTAND:
    model.addCons(
        quicksum(säule_hat_typ[b, s] for s in STypen) <= 1
        )

In [35]:
#!!

# NB 13: Max. dürfen drei Ladesäulen angeschafft werden und Anzahl der angeschafften Säulen im Bestand = Anzahl der Säulen
model.addCons(
    quicksum(anzahl_Säule[s] for s in STypen) <= 3
    )

# brauchen wir das?
for s in STypen:
    model.addCons(
        anzahl_Säule[s] == quicksum(säule_hat_typ[b, s] for b in SÄULENBESTAND)
        )


In [36]:
# NB: Ladesäulen nur anschaffen, wenn auch E-LKWs vorhanden sind (Big M Logik)

M = 10 # Eine ausreichend große Zahl, da max 3 Säulen erlaubt sind
model.addCons(
    quicksum(anzahl_Säule[s] for s in STypen) <= M * quicksum(anzahl_lkw[t] for t in ["E400", "E600"]),
    name="Ladesaeulen_nur_mit_ELKW"
    )

Ladesaeulen_nur_mit_ELKW

In [37]:
# NB : Max. 2 Ladepunkte pro Säule:
#for z in ZEIT:
 #   for b in SÄULENBESTAND:
  #      model.addCons(
   #         quicksum(lkw_lädt[f, z, b, l] for f in FUHRPARK for l in LADEPUNKT) <= 2 * quicksum(säule_hat_typ[b ,s] for s in STypen),
    #        name=f"MaxLadepunkte_{z}_{s}"
     #   )

*Wer darf wo laden?*

In [38]:
# !!

# LKW-Sicht
# NB 14: Ein LKW kann zur Zeit z maximal an EINER Säule laden
for z in ZEIT:
  for b in SÄULENBESTAND:
      for l in LADEPUNKT:
        model.addCons(
            quicksum(lkw_lädt[f, z, b, l] for f in FUHRPARK) <= 1
            )

In [39]:
#!!

# Ladepunkte-Sicht
# NB: An einem Ladepunkt darf max ein LKW gleichzeitig laden
for f in FUHRPARK:
    for z in ZEIT:
        model.addCons(
            quicksum(lkw_lädt[f, z, b, l] for b in SÄULENBESTAND for l in LADEPUNKT) <= 1,
        )

In [40]:
#!!

# LKW-Sicht
# NB 7: LKW nur laden, wenn er nicht auf Tour ist - Zeitliche Überlappung verhindern
for f in FUHRPARK:
    for z in ZEIT:
        # Sum of charging activity for truck f at time z
        charging_activity = quicksum(lkw_lädt[f, z, b, l] for b in SÄULENBESTAND for l in LADEPUNKT)
        # Sum of active route participation for truck f at time z
        # This sum will be 1 if the truck is on ANY active route at time z, and 0 otherwise
        on_tour_activity = quicksum(lkw_fährt_r[f, r] * ROUTE_AKTIV[r, z] for r in ROUTEN)

        model.addCons(
            charging_activity <= 1 - on_tour_activity,
            name=f"NoChargingWhileOnTour_{f}_{z}"
        )

In [41]:
# NB 8: Laden wird nur für die ELKW erlaubt, Diesel darf nicht laden
for f in FUHRPARK:
    for z in ZEIT:
        for d in SÄULENBESTAND:
          for l in LADEPUNKT:
            model.addCons(lkw_lädt[f, z, d, l] <= lkw_hat_typ[f, "E400"] + lkw_hat_typ[f, "E600"])

In [42]:
# brauchen wir das?
# 9 Ladepunkte-Limit: Anzahl der geladenen LKWs darf Anzahl Ladepunkte nicht überschreiten
for z in ZEIT:
        model.addCons(
            quicksum(lkw_lädt[f, z, d, l] for f in FUHRPARK for b in SÄULENBESTAND for l in LADEPUNKT) <= 2 * quicksum(anzahl_Säule[s] for l in LADEPUNKT)
            )

In [43]:
# Kopplung: Ladevorgang nur möglich, wenn die physische Säule b einen Typ s zugewiesen bekommen hat
for z in ZEIT:
    for b in SÄULENBESTAND:
        model.addCons(
            quicksum(lkw_lädt[f, z, b, l] for f in FUHRPARK for l in LADEPUNKT) <=
            quicksum(säule_hat_typ[b, s] for s in STypen),
            name=f"Kopplung_Laden_Saeule_{z}_{b}"
        )

*Kopplung Logik und Physik*

In [44]:
# !!

# NB: Leistung kann nur fließen, wenn lkw_lädt[f,z, d, l] = 1   (= Kopplung leistung_lkw_von_ladepunkt an lkw_lädt)
for f in FUHRPARK:
    for z in ZEIT:
        for b in SÄULENBESTAND:
            for l in LADEPUNKT:
              model.addCons(
                    leistung_lkw_von_ladepunkt[f, z, b, l] <= max(POWER_LIMIT_S.values()) * lkw_lädt[f, z, b, l],
                    name=f"PowerIfCharging_{f}_{z}_{b}_{l}"
                )


*Batterieleistung -> wie schnell laden*

In [45]:
#!!

# Gesamte Sicht: von beiden Ladepunkten <= max was Säule liefern kann
# NB: PL_Säule: Max. Kapazität der Säule: Die Summe der abgerufenen Leistung (p_load) aller LKWs an dieser Säule darf die Gesamtkapazität der gekauften Säulen (n * PL_S) nicht überschreiten
for z in ZEIT:
    for b in SÄULENBESTAND:
        model.addCons(
            quicksum(leistung_ladepunkt_von_säule[z, b, l] for l in LADEPUNKT) <=
            quicksum(säule_hat_typ[b, s] * POWER_LIMIT_S[s] for s in STypen),
            name=f"SaeulenKapazitaet_{z}_{b}"
        )

In [46]:
# einzelne Sichteweise pro Ladepunkt
# LKW darf an diesem Ladepunkt (b,l) zur Zeit z nicht mehr Leistung ziehen, als der Ladepunkt maximal liefern kann.
#for b in SÄULENBESTAND:
 #   for l in LADEPUNKT:
  #      for z in ZEIT:
   #         model.addCons(
    #            quicksum(leistung_lkw_von_ladepunkt[f, z, b, l] for f in FUHRPARK) <= leistung_ladepunkt_von_säule[z, b, l],
     #           name=f"FlussLimit_{z}_{b}_{l}"
      #      )

In [47]:
# NB: PL_LKW: Die abgerufene Leistung darf das Lade-Limit des LKW-Typs nicht überschreiten
for f in FUHRPARK:
    for z in ZEIT:
        for b in SÄULENBESTAND:
            for l in LADEPUNKT:
                model.addCons(
                    leistung_lkw_von_ladepunkt[f, z, b, l] <= quicksum(lkw_hat_typ[f, t] * POWER_LIMIT_LKW[t] for t in FTypen),
                    name=f"LKW_FahrzeugLimit_{f}_{z}_{b}_{l}"
                )

*Batteriekapazität -> wie viel Platz haben wir in der Batterie*

In [48]:
# NB: Der SOC am Anfang (Schritt 0) muss exakt dem SOC am Ende (Schritt 95) entsprechen -> Das ist deine Bedingung: "Tag 2 Zeit 0 == Tag 1 Zeit 95"
for f in FUHRPARK:
    model.addCons(
        soc[f, 0] == soc[f, 95],
        #soc[f, 0] == quicksum(lkw_hat_typ[f, t] * BATTERIE[t] for t in FTypen) * 0.8,
        name=f"Zyklus_SOC_LKW_{f}")

In [49]:
# NB 10: Reichweiten-Check & Ladepflicht
for f in FUHRPARK:
    for r in ROUTEN:
        # Startzeitpunkt der Route bestimmen (erster Zeitschritt mit U_rz == 1)
        start_schritte = [z for z in ZEIT if ROUTE_AKTIV[r, z] > 0.5]
        if start_schritte:

          z_start = min(start_schritte)

          for t in FTypen:
            if t != "Diesel":
                # Benötigte Energie für die gesamte Tour r
                bedarf = DISTANZ_GESAMT[r] * VERBRAUCH[t]

                # Wenn LKW f vom Typ t ist und Tour r fährt (z_frt == 1),
                # dann MUSS der SOC am Startpunkt >= Bedarf sein.
                # Wir nutzen ein "Big M" (hier max(BATTERIE.values())), um die Bedingung zu aktivieren:
                model.addCons(
                    soc[f, z_start] >= bedarf - (1 - lkw_fährt_Route_mit_typ[f, r, t]) * max(BATTERIE.values()),
                    name=f"Check_Reichweite_LKW{f}_Tour{r}"
                )

# NB 12: Erzwungenes Laden (Zusatz-Logik)
# Damit der LKW nicht "vergisst" zu laden, wenn er Zeit hat:
for f in FUHRPARK:
    for z in ZEIT:
        # Wenn der LKW ein E-LKW ist, darf er nie unter 10% (Sicherheitsreserve)
        model.addCons(
            soc[f, z] >= 0.1 * quicksum(lkw_hat_typ[f, t] * BATTERIE[t] for t in ["E400","E600"]),
            name=f"Sicherheitsreserve_f{f}_z{z}"
        )

In [50]:
# NB: Kapazitätsgrenze: SOC darf nie über die Batteriekapazität steigen
for f in FUHRPARK:
  for z in ZEIT:
        model.addCons(soc[f, z] <= quicksum(lkw_hat_typ[f, t] * BATTERIE[t] for t in FTypen))

In [51]:
# NB 9: Energiefluss (SOC) für einen repräsentativen Standardtag (96 Schritte)
for f in FUHRPARK:
    for z in ZEIT:
        if z > 0:
            # 1. VERBRAUCH (Entnahme)
            entnahme = quicksum(
                lkw_fährt_Route_mit_typ[f, r, t] * ((DISTANZ_GESAMT[r] * VERBRAUCH[t]) / ROUTE_DAUER[r])
                for r in ROUTEN for t in FTypen if t != "Diesel"
            )

            # 2. LADUNG (Zufuhr)
            zufuhr = quicksum(leistung_lkw_von_ladepunkt[f, z, b, l] for b in SÄULENBESTAND for l in LADEPUNKT) / 4

            # 3. ENERGIEBILANZ
            model.addCons(
                soc[f, z] == soc[f, z-1] - entnahme + zufuhr,
                name=f"SOC_Fluss_f{f}_z{z}")

In [52]:
# NB: Erzwungene volle Auslastung zu jedem Zeitpunkt z (es hängt immer ein LKW an der Ladesäule)
#for z in ZEIT:
 #   for s in STypen:
  #      # Die Summe der ladenden LKWs darf höchstens der Anzahl der Ladepunkte entsprechen
   #     model.addCons(
    #        quicksum(lkw_lädt[f, z, s] for f in FUHRPARK) <= quicksum(anzahl_Säule[s] for l in LADEPUNKT),
     #       name=f"Full_Utilization_z{z}_s{s}"
      #  )

Modell optimieren

In [ ]:
#model.addCons(anzahl_lkw["E400"] + anzahl_lkw["E600"] >= 1, name="Force_One_ELKW")

In [ ]:
model.setRealParam("limits/time", 3800) # Gib ihm 5 Minuten Zeit
model.optimize()
print(f"Solver Status: {model.getStatus()}")

Ausgabe

In [ ]:
if model.getNSols() > 0:
    print("\n" + "="*50)
    print(f"ERGEBNIS: {model.getObjVal():,.2f} € Gesamtkosten/Jahr")
    print("="*50)

    try:
        print("\n--- SOLVER-STATUS ---")
        print("Status:", model.getStatus())
        gap = model.getGap()
        optimalitaet = (gap) * 100
        print(f"Optimalität: {optimalitaet:.2f}%")
        print("Solving time (s):", model.getSolvingTime())
        print("Nodes:", model.getNNodes())
    except Exception:
        pass

    # 1. ANZAHL UND TYP DER LKWS
    print("\n--- FLOTTENZUSAMMENSETZUNG (anzahl_lkw) ---")
    for t in FTypen:
        anzahl = int(model.getVal(anzahl_lkw[t]))
        if anzahl > 0:
            print(f"➤ {t:8}: {anzahl} Fahrzeug(e)")

    # 2. ZUORDNUNG DER TOUREN (lkw_fährt_r und lkw_hat_typ)
    print("\n--- EINSATZPLAN PRO LKW-SLOT (lkw_fährt_r und lkw_hat_typ) ---")
    gefundene_lkws = 0
    for f in FUHRPARK:
        lkw_typ = next((t for t in FTypen if model.getVal(lkw_hat_typ[f, t]) > 0.5), None)
        if lkw_typ:
            tours_for_f = [routes_df.loc[r, 'route_id'] for r in ROUTEN if model.getVal(lkw_fährt_r[f, r]) > 0.5]
            if tours_for_f:
                gefundene_lkws += 1
                tour_liste = ", ".join(tours_for_f)
                print(f"[{lkw_typ:6}]: LKW {f} fährt Tour(en) -> {tour_liste}")

    # NEW SECTION: DETAILED TRUCK-ROUTE-TYPE ASSIGNMENT (lkw_fährt_Route_mit_typ)
    print("\n--- DETAILLIERTER LKW-ROUTEN-TYP PLAN (lkw_fährt_Route_mit_typ) ---")
    detailed_assignment_found = False
    for f in FUHRPARK:
        for r in ROUTEN:
            for t in FTypen:
                if model.getVal(lkw_fährt_Route_mit_typ[f, r, t]) > 0.5:
                    detailed_assignment_found = True
                    print(f"LKW {f} (Typ: {t}) fährt Route {routes_df.loc[r, 'route_id']}")
    if not detailed_assignment_found:
        print("Keine detaillierten LKW-Routen-Typ-Zuordnungen gefunden.")

    # 3. BESCHAFFTE LADEINFRASTRUKTUR (anzahl_Säule)
    print("\n--- LADEINFRASTRUKTUR (anzahl_Säule) ---")
    infrastruktur_vorhanden = False
    for s in STypen:
        anzahl_s = int(round(model.getVal(anzahl_Säule[s])))
        if anzahl_s > 0:
            infrastruktur_vorhanden = True
            print(f"➤ {s:15}: {anzahl_s} Säule(n) beschafft")
    if not infrastruktur_vorhanden:
        print(" Keine Ladesäulen benötigt (reiner Diesel-Betrieb).")

    # NEW SECTION: ASSIGNMENT OF CHARGING POLES TO TYPES (säule_hat_typ)
    print("\n--- ZUORDNUNG SÄULENBESTAND ZU TYPEN (säule_hat_typ) ---")
    pole_type_assignment_found = False
    for b in SÄULENBESTAND:
        for s in STypen:
            if model.getVal(säule_hat_typ[b, s]) > 0.5:
                pole_type_assignment_found = True
                print(f"Säule {b} ist vom Typ: {s}")
    if not pole_type_assignment_found:
        print("Keine Säulen im Bestand zugewiesen.")

    # NEW SECTION: CHARGING ACTIVITIES AND POWER FLOWS (lkw_lädt, leistung_ladepunkt_von_säule, leistung_lkw_von_ladepunkt)
    print("\n--- DETAILLIERTE LADEAKTIVITÄTEN UND ENERGIEFLÜSSE (lkw_lädt, leistung_ladepunkt_von_säule, leistung_lkw_von_ladepunkt) ---")
    charging_activity_found_overall = False
    for z in ZEIT:
        time_has_activity = False
        for f in FUHRPARK:
            for b in SÄULENBESTAND:
                for l in LADEPUNKT:
                    if model.getVal(lkw_lädt[f, z, b, l]) > 0.5:
                        charging_activity_found_overall = True
                        time_has_activity = True
                        lkw_typ = next((t for t in FTypen if model.getVal(lkw_hat_typ[f, t]) > 0.5), "N/A")
                        leistung_lkw = model.getVal(leistung_lkw_von_ladepunkt[f, z, b, l])
                        leistung_ladepunkt_at_slot = model.getVal(leistung_ladepunkt_von_säule[z, b, l]) # Corrected variable name
                        uhrzeit = f"{int((z*15)//60):02d}:{int((z*15)%60):02d}"
                        print(f"[{uhrzeit}] LKW {f} (Typ: {lkw_typ}) lädt an Säule {b}, Ladepunkt {l} mit {leistung_lkw:.1f} kW (Ladepunkt {l} an Säule {b} liefert: {leistung_ladepunkt_at_slot:.1f} kW).")
        if time_has_activity:
            print("-" * 100) # Separator for different time steps with activity
    if not charging_activity_found_overall:
        print("Keine Ladeaktivitäten oder Energieflüsse registriert.")

    # Original section for SOC and activity timeline
    print("\n--- ZEITLICHER VERLAUF DER LKW-AKTIVITÄT & BATTERIESTAND (soc) ---")
    print("Diese detaillierte Ansicht zeigt den Status jedes LKWs pro Zeitintervall.")
    print("\n" + "="*80)
    print(f"{'Uhrzeit':<8} | {'LKW':<8} | {'Typ':<8} | {'Aktivität':<20} | {'Batterie'}")
    print("-" * 80)

    for f in FUHRPARK:
        lkw_typ = next((t for t in FTypen if model.getVal(lkw_hat_typ[f, t]) > 0.5), None)
        if not lkw_typ: continue

        last_status_display = None
        last_soc_display = None
        for z in ZEIT:
            uhrzeit = f"{int((z*15)//60):02d}:{int((z*15)%60):02d}"
            soc_val = model.getVal(soc[f, z])

            # Status-Check
            current_status = "Standby"
            for r in ROUTEN:
                if model.getVal(lkw_fährt_r[f, r]) > 0.5 and ROUTE_AKTIV[r, z] > 0.5:
                    current_status = f"🚚 Tour {routes_df.loc[r, 'route_id']}"
                    break
            if current_status == "Standby":
                for b_idx in SÄULENBESTAND:
                    for l_idx in LADEPUNKT:
                        if model.getVal(lkw_lädt[f, z, b_idx, l_idx]) > 0.5:
                            current_status = f"⚡ Laden an S.{b_idx}/LP.{l_idx}" # More specific charging location
                            break
                    if current_status != "Standby":
                        break

            # Only print if status or soc changed significantly
            # Adding a small epsilon to check for significant SOC change to avoid printing for minor numerical fluctuations
            if current_status != last_status_display or (last_soc_display is not None and abs(soc_val - last_soc_display) > 0.1):
                b_anzeige = f"{soc_val:6.1f} kWh" if lkw_typ != "Diesel" else "---"
                print(f"{uhrzeit:<8} | LKW {f:02d} | {lkw_typ:<8} | {current_status:<20} | {b_anzeige}")
                last_status_display = current_status
                last_soc_display = soc_val
        print("-" * 80) # Separator for each truck's timeline

else:
    print("Keine Lösung gefunden.")